##                                                Entraînement du Modéle IA
### Importation des modules 

















 























In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import joblib


### A - Chargement de données d'entraînement

In [2]:
# Charger le dataset
data = pd.read_csv('train.csv')

# Afficher les premières lignes du dataset
data.head()


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Nous allons supprimer les colonnes qui sont unitile pour l'entrainement de notre modele. Dans notre dataset nous juste la colonne name qui est unitile. 

In [3]:
data.drop(columns=["Name"], inplace=True)

### B - Préparation des données

#### 1- Gestion des variables manquantes
Nous allons d'abord vérifier les variables manquantes dans les données en utilisant isnull().

In [4]:
data.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Transported       0
dtype: int64

In [ ]:
for col in ['HomePlanet', 'Destination', 'VIP']:
    data[col] = data[col].fillna(data[col].mode()[0])

data['Age'] = data['Age'].fillna(data['Age'].median())


cols_depenses = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
data.loc[data['CryoSleep'].isna() & (data[cols_depenses].sum(axis=1) > 0), 'CryoSleep'] = False




for col in cols_depenses:
    mediane = data[col].median()
    data[col] = data[col].fillna(mediane)
data.isnull().sum()

colonnes_cibles = ["CryoSleep", "Cabin"]
data_cleaned = data.dropna(subset=colonnes_cibles)

data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8398 entries, 0 to 8692
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8398 non-null   object 
 1   HomePlanet    8398 non-null   object 
 2   CryoSleep     8398 non-null   object 
 3   Cabin         8398 non-null   object 
 4   Destination   8398 non-null   object 
 5   Age           8398 non-null   float64
 6   VIP           8398 non-null   bool   
 7   RoomService   8398 non-null   float64
 8   FoodCourt     8398 non-null   float64
 9   ShoppingMall  8398 non-null   float64
 10  Spa           8398 non-null   float64
 11  VRDeck        8398 non-null   float64
 12  Transported   8398 non-null   bool   
dtypes: bool(2), float64(6), object(5)
memory usage: 803.7+ KB


/tmp/ipykernel_16133/2262445677.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data[col] = data[col].fillna(data[col].mode()[0])


#### 2- Transformation des données

In [6]:
data_cleaned['CryoSleep'] = data_cleaned['CryoSleep'].astype(bool)


data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8398 entries, 0 to 8692
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8398 non-null   object 
 1   HomePlanet    8398 non-null   object 
 2   CryoSleep     8398 non-null   bool   
 3   Cabin         8398 non-null   object 
 4   Destination   8398 non-null   object 
 5   Age           8398 non-null   float64
 6   VIP           8398 non-null   bool   
 7   RoomService   8398 non-null   float64
 8   FoodCourt     8398 non-null   float64
 9   ShoppingMall  8398 non-null   float64
 10  Spa           8398 non-null   float64
 11  VRDeck        8398 non-null   float64
 12  Transported   8398 non-null   bool   
dtypes: bool(3), float64(6), object(4)
memory usage: 746.3+ KB


/tmp/ipykernel_16133/1607860048.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned['CryoSleep'] = data_cleaned['CryoSleep'].astype(bool)


In [7]:
data_cleaned = data_cleaned.copy()

data_cleaned['GroupId'] = data_cleaned['PassengerId'].apply(lambda x: x.split('_')[0])
group_sizes = data_cleaned['GroupId'].value_counts()
data_cleaned['GroupSize'] = data_cleaned['GroupId'].map(group_sizes)
data_cleaned.drop(["GroupId", "PassengerId"], axis=1, inplace=True)

data_cleaned[['Deck','Cabin_Num','Side']]=data_cleaned['Cabin'].str.split("/",expand = True)
data_cleaned.drop(['Cabin'], axis=1, inplace=True)

cols_depenses = ['RoomService','FoodCourt','Spa','VRDeck']
data_cleaned['TotalSpend'] = data_cleaned[cols_depenses].sum(axis =1)

data_cleaned.head()

colonne =  ["CryoSleep","VIP","Transported","Cabin_Num"]
data_cleaned[colonne] = data_cleaned[colonne].astype(int)

data_cleaned.info()
colonne =  ["Cabin_Num","GroupSize"]
data_cleaned[colonne] = data_cleaned[colonne].astype(float)

data_cleaned.info()


<class 'pandas.core.frame.DataFrame'>
Index: 8398 entries, 0 to 8692
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    8398 non-null   object 
 1   CryoSleep     8398 non-null   int64  
 2   Destination   8398 non-null   object 
 3   Age           8398 non-null   float64
 4   VIP           8398 non-null   int64  
 5   RoomService   8398 non-null   float64
 6   FoodCourt     8398 non-null   float64
 7   ShoppingMall  8398 non-null   float64
 8   Spa           8398 non-null   float64
 9   VRDeck        8398 non-null   float64
 10  Transported   8398 non-null   int64  
 11  GroupSize     8398 non-null   int64  
 12  Deck          8398 non-null   object 
 13  Cabin_Num     8398 non-null   int64  
 14  Side          8398 non-null   object 
 15  TotalSpend    8398 non-null   float64
dtypes: float64(7), int64(5), object(4)
memory usage: 1.1+ MB
<class 'pandas.core.frame.DataFrame'>
Index: 8398 entries, 0 t

## C - Encodage et normalisation
### 1 - Encodage des variables categorielles

In [8]:
categorical_cols = ['HomePlanet', 'Destination', 'Deck', 'Side']
data_encoded = pd.get_dummies(data_cleaned, columns=categorical_cols, drop_first=False)

bool_columns = data_encoded.select_dtypes(include='bool').columns
data_encoded[bool_columns] = data_encoded[bool_columns].astype(int)

data_encoded.head()

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,GroupSize,...,Deck_A,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_T,Side_P,Side_S
0,0,39.0,0,0.0,0.0,0.0,0.0,0.0,0,1.0,...,0,1,0,0,0,0,0,0,1,0
1,0,24.0,0,109.0,9.0,25.0,549.0,44.0,1,1.0,...,0,0,0,0,0,1,0,0,0,1
2,0,58.0,1,43.0,3576.0,0.0,6715.0,49.0,0,2.0,...,1,0,0,0,0,0,0,0,0,1
3,0,33.0,0,0.0,1283.0,371.0,3329.0,193.0,0,2.0,...,1,0,0,0,0,0,0,0,0,1
4,0,16.0,0,303.0,70.0,151.0,565.0,2.0,1,1.0,...,0,0,0,0,0,1,0,0,0,1


### 2 - Normalisation des variables numeriques

In [9]:
numeric_cols = [
    'Age', 'RoomService', 'FoodCourt', 'ShoppingMall',
    'Spa', 'VRDeck', 'Cabin_Num', 'GroupSize', 'TotalSpend'
]

scaler = StandardScaler()
data_encoded[numeric_cols] = scaler.fit_transform(data_encoded[numeric_cols])

data_encoded.head()

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,GroupSize,...,Deck_A,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_T,Side_P,Side_S
0,0,0.710496,0,-0.344553,-0.281835,-0.284999,-0.272519,-0.264976,0,-0.641321,...,0,1,0,0,0,0,0,0,1,0
1,0,-0.337687,0,-0.173698,-0.276242,-0.243673,0.220408,-0.226488,1,-0.641321,...,0,0,0,0,0,1,0,0,0,1
2,0,2.038193,1,-0.277152,1.940425,-0.284999,5.756635,-0.222114,0,0.015005,...,1,0,0,0,0,0,0,0,0,1
3,0,0.291223,0,-0.344553,0.515469,0.328267,2.716469,-0.096154,0,0.015005,...,1,0,0,0,0,0,0,0,0,1
4,0,-0.896717,0,0.130392,-0.238334,-0.035395,0.234774,-0.263226,1,-0.641321,...,0,0,0,0,0,1,0,0,0,1


Je récupère le scaler utilisé pour la normalisation afin de l'utiliser dans mon interface Streamlit.

In [10]:
colonnes_modele = data_encoded.columns.tolist()
joblib.dump(colonnes_modele, "colonnes_modele.joblib")

joblib.dump(scaler , "scarler2.joblib")

['scarler2.joblib']

Je sépare les variables caractéristiques de la variable cible.


In [11]:
cible = data_encoded['Transported']
carac = data_encoded.drop(columns=['Transported'], axis=1) 

# D - Modèles
Les données ont été divisées en un jeu d'entraînement et un jeu de test pour évaluer les performances du modèle.

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(carac, cible, test_size=0.3, random_state=42)


### 1- Régerssion Logistique

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve



log_reg = LogisticRegression(random_state=42)
log_reg.fit(X_train, y_train)

y_pred_log_reg = log_reg.predict(X_test)

# Matrice de confusion
conf_matrix_log_reg = confusion_matrix(y_test, y_pred_log_reg)
print("Matrice de Confusion (Régression Logistique) :\n", conf_matrix_log_reg)

# Rapport de classification
class_report_log_reg = classification_report(y_test, y_pred_log_reg)
print("Rapport de Classification (Régression Logistique) :\n", class_report_log_reg)

# AUC-ROC
roc_auc_log_reg = roc_auc_score(y_test, y_pred_log_reg)
print("AUC-ROC (Régression Logistique) :\n", roc_auc_log_reg)

Matrice de Confusion (Régression Logistique) :
 [[ 991  247]
 [ 258 1024]]
Rapport de Classification (Régression Logistique) :
               precision    recall  f1-score   support

           0       0.79      0.80      0.80      1238
           1       0.81      0.80      0.80      1282

    accuracy                           0.80      2520
   macro avg       0.80      0.80      0.80      2520
weighted avg       0.80      0.80      0.80      2520

AUC-ROC (Régression Logistique) :
 0.7996183013717963


### 2 - Decision Tree

In [14]:
from sklearn.tree import DecisionTreeClassifier

# Initialisation et entraînement du modèle
tree_model = DecisionTreeClassifier(random_state=42)
tree_model.fit(X_train, y_train)

# Prédictions
y_pred_tree = tree_model.predict(X_test)

# Matrice de confusion
conf_matrix_tree = confusion_matrix(y_test, y_pred_tree)
print("Matrice de Confusion (Decision Tree) :\n", conf_matrix_tree)

# Rapport de classification
class_report_tree = classification_report(y_test, y_pred_tree)
print("Rapport de Classification (Decision Tree) :\n", class_report_tree)

# AUC-ROC
roc_auc_tree = roc_auc_score(y_test, y_pred_tree)
print("AUC-ROC (Decision Tree) :\n", roc_auc_tree)

Matrice de Confusion (Decision Tree) :
 [[949 289]
 [323 959]]
Rapport de Classification (Decision Tree) :
               precision    recall  f1-score   support

           0       0.75      0.77      0.76      1238
           1       0.77      0.75      0.76      1282

    accuracy                           0.76      2520
   macro avg       0.76      0.76      0.76      2520
weighted avg       0.76      0.76      0.76      2520

AUC-ROC (Decision Tree) :
 0.7573044440355966


### 3- Support Vector Machine (SVM)


In [15]:
from sklearn.svm import SVC


svm = SVC(probability=True)
svm.fit(X_train, y_train)


y_pred_svm = svm.predict(X_test)


# Matrice de confusion
conf_matrix_svm = confusion_matrix(y_test, y_pred_svm)
print("Matrice de Confusion :\n", conf_matrix_svm)

# Rapport de classification
class_report_svm = classification_report(y_test, y_pred_svm)
print("Rapport de Classification :\n", class_report_svm)

# AUC-ROC
roc_auc_svm = roc_auc_score(y_test, y_pred_svm)
print("AUC-ROC :\n", roc_auc_svm)

Matrice de Confusion :
 [[1025  213]
 [ 257 1025]]
Rapport de Classification :
               precision    recall  f1-score   support

           0       0.80      0.83      0.81      1238
           1       0.83      0.80      0.81      1282

    accuracy                           0.81      2520
   macro avg       0.81      0.81      0.81      2520
weighted avg       0.81      0.81      0.81      2520

AUC-ROC :
 0.8137401424974607


### 4 - Perceptron

In [16]:
from sklearn.linear_model import Perceptron


perceptron = Perceptron(max_iter=1000, random_state=42, tol=1e-3)
perceptron.fit(X_train, y_train)

y_pred_perceptron = perceptron.predict(X_test)

# Matrice de confusion
conf_matrix_perceptron = confusion_matrix(y_test, y_pred_perceptron)
print("Matrice de Confusion \n", conf_matrix_perceptron)

# Rapport de classification
class_report_perceptron = classification_report(y_test, y_pred_perceptron)
print("Rapport de Classification:\n", class_report_perceptron)

# AUC-ROC
roc_auc_perceptron = roc_auc_score(y_test, y_pred_perceptron)
print("AUC-ROC :\n", roc_auc_perceptron)

Matrice de Confusion 
 [[905 333]
 [409 873]]
Rapport de Classification:
               precision    recall  f1-score   support

           0       0.69      0.73      0.71      1238
           1       0.72      0.68      0.70      1282

    accuracy                           0.71      2520
   macro avg       0.71      0.71      0.71      2520
weighted avg       0.71      0.71      0.71      2520

AUC-ROC :
 0.7059925046436429


### 5 -Naive Bayès


In [17]:
from sklearn.naive_bayes import GaussianNB


nb = GaussianNB()
nb.fit(X_train, y_train)


y_pred_nb = nb.predict(X_test)

# Matrice de confusion
conf_matrix_nb = confusion_matrix(y_test, y_pred_nb)
print("Matrice de Confusion (Naive Bayes) :\n", conf_matrix_nb)

# Rapport de classification
class_report_nb = classification_report(y_test, y_pred_nb)
print("Rapport de Classification (Naive Bayes) :\n", class_report_nb)

# AUC-ROC
roc_auc_nb = roc_auc_score(y_test, y_pred_nb)
print("AUC-ROC (Naive Bayes) :\n", roc_auc_nb)


Matrice de Confusion (Naive Bayes) :
 [[ 691  547]
 [ 127 1155]]
Rapport de Classification (Naive Bayes) :
               precision    recall  f1-score   support

           0       0.84      0.56      0.67      1238
           1       0.68      0.90      0.77      1282

    accuracy                           0.73      2520
   macro avg       0.76      0.73      0.72      2520
weighted avg       0.76      0.73      0.72      2520

AUC-ROC (Naive Bayes) :
 0.7295471786561285


### 6. Random Forest

In [18]:
from sklearn.ensemble import RandomForestClassifier


rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)


y_pred_rf = rf_model.predict(X_test)

# Matrice de confusion
conf_matrix_rf = confusion_matrix(y_test, y_pred_rf)
print("Matrice de Confusion (Random Forest) :\n", conf_matrix_rf)

# Rapport de classification
class_report_rf = classification_report(y_test, y_pred_rf)
print("Rapport de Classification (Random Forest) :\n", class_report_rf)

# AUC-ROC
roc_auc_rf = roc_auc_score(y_test, y_pred_rf)
print("AUC-ROC (Random Forest) :\n", roc_auc_rf)

Matrice de Confusion (Random Forest) :
 [[1074  164]
 [ 300  982]]
Rapport de Classification (Random Forest) :
               precision    recall  f1-score   support

           0       0.78      0.87      0.82      1238
           1       0.86      0.77      0.81      1282

    accuracy                           0.82      2520
   macro avg       0.82      0.82      0.82      2520
weighted avg       0.82      0.82      0.82      2520

AUC-ROC (Random Forest) :
 0.8167594555155387


### 7 - Modèle final
Notre modèle final est le svm et on l'exporte  avec `joblib`.

In [19]:
joblib.dump(svm , "modele_svm.joblib")

['modele_svm.joblib']